In [2]:
from ipywidgets import HTML,Text,HBox,VBox,Button,Dropdown
from IPython.display import display
!pip install gensim
from gensim.models import Word2Vec
model = Word2Vec.load("gr10.vec")

In [68]:
display(HTML('''
<style>
.tbl {width:80%}
</style>
'''))


HTML(value='\n<style>\n.tbl {width:80%}\n</style>\n')

In [69]:
def expl(c):
    W.value='πόνος'
    gosim(c)

In [70]:
import glob
mchoice=Dropdown(options=glob.glob('*.vec'), description="Model size", value='gr10.vec')

def on_change(change):
    global model
    if change['type'] == 'change' and change['name'] == 'value':
        model=Word2Vec.load(change['new'])

mchoice.observe(on_change)



In [71]:
def gosim(c):
    try:
        r=model.wv.most_similar(W.value, topn=20)
        output="<table class='tbl'>"
        for m,s in r:
            output+=f"<tr><td>{m}</td><td>{s:.2f}</td></tr>"
        output+='</table>'
    except:
        output="Unknown word"
    Out.value=output

In [72]:
W=Text(description="Word in greek")
B=Button(description="Similarity")
Be=Button(description="Example")
B.on_click(gosim)
Be.on_click(expl)
Out=HTML("")


In [73]:
def infer(c):
    try:
        r=model.wv.most_similar(positive=positive.value.split(' '), negative=negative.value.split(' '))
        output="<table class='tbl'>"
        for m,s in r:
            output+=f"<tr><td>{m}</td><td>{s:.2f}</td></tr>"
        output+='</table>'
    except:
        output="Some words are unknown"
    Out.value=output
    
def expl2(c):
    positive.value="πόνος γυνή"
    negative.value="ἀνήρ"
    infer(c)

    

In [74]:
positive=Text(description="Positive")
negative=Text(description="Negative")
BB=Button(description="Infer")
BBe=Button(description='example')
Out2=HTML("")
BB.on_click(infer)
BBe.on_click(expl2)


In [75]:
VBox([mchoice,HTML("<h1>Word similarity</h1>"),
      VBox([HBox([W,B,Be])]),
      HTML("<h1>Sense addition</h1>"),
      VBox([HBox([positive,negative]),HBox([BB,BBe]),Out])])